In [29]:
import pandas as pd

from src.dataset import DepressionDataset
from src.train import train
from src.model import GCN

In [30]:
raw_train = pd.read_table("./data/bronze/train.tsv", sep="\t").groupby("label").head(2000).reset_index(drop=True)
raw_dev = pd.read_table("./data/bronze/dev.tsv", sep="\t")

In [31]:
dataset_path = './data/gold'
dataset_root = dataset_path + "/w2v_window_11"

train_dataset = DepressionDataset(
    root=dataset_root, filename="",
    prefix="train",
    raw_data=raw_train
)

dev_dataset = DepressionDataset(
    root=dataset_root, filename="",
    prefix="dev",
    raw_data=raw_dev
)

In [32]:
HYPERPARAMETERS = {
    "n_epochs": [500],
    "batch_size": [32, 128, 64],
    "learning_rate": [0.1, 0.05, 0.01, 0.001],
    "weight_decay": [0.0001, 0.00001, 0.001],
    "sgd_momentum": [0.9, 0.8, 0.5],
    "scheduler_gamma": [0.995, 0.9, 0.8, 0.5, 1],
    "model_embedding_size": [8, 16, 32, 64, 128],
    "model_dense_neurons": [16, 128, 64, 256, 32],
    "model_layers": [3],
    "model_num_classes": [3],
}

In [34]:
from mango import Tuner

config = dict()
config["optimizer"] = "Bayesian"
config["num_iteration"] = 100

tuner = Tuner(HYPERPARAMETERS, 
              objective=lambda params: train(params[0], train_dataset, dev_dataset, GCN),
              conf_dict=config) 
results = tuner.minimize()

100%|██████████| 36/36 [00:06<00:00,  5.96it/s]
